# Testing speed of the tree algorithm

# Implement a simple network

In [5]:
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

sys.path.append('../')

from src_experiment import NeuralNet
from geobin import RegionTree, TreeNode

In [12]:
net = NeuralNet(
    input_size = 2,
    hidden_sizes = [7,5,4,3],
    num_classes = 2,
)
state_dict = net.state_dict()

In [13]:
tree = RegionTree(state_dict)
tree.build_tree(verbose=True)

Building tree...


Layer 5: 100%|██████████| 4/4 [00:39<00:00,  9.98s/it]
